In [ ]:
print("ok")

pip3 install pymilvus==2.4.3

In [1]:
pip install pymilvus

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
CLOUD_REGION = 'gcp-us-west1'
CLUSTER_ID = 'in03-22d9784f7842941'
API_KEY = '261e4b2aa8e7db717791e0db55c82ec2c7c42593c07c9f60e9ba70bdbcbfa5a12f0133c7aead468155c5b3317cc8fc543266a1fd'
PROJECT_ID = 'proj-883eb41213c060ba3953b1'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
print(extracted_data[0].page_content)

.., 
GI 
C 
,a 
0.


In [7]:
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.zilliz import Zilliz
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.chat_models import ChatOpenAI
#from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
#import sentence_transformers
#from langchain.embeddings import HuggingFaceEmbeddings

import os

# 1. Set up the name of the collection to be created.
COLLECTION_NAME = 'my_text_collection'

# 2. Set up the dimension of the embeddings.
DIMENSION = 768

# 3. Set up the cohere api key
# OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 4. Set up the connection parameters for your Zilliz Cloud cluster.
URI = 'https://in03-22d9784f7842941.serverless.gcp-us-west1.cloud.zilliz.com'

# 5. Set up the token for your Zilliz Cloud cluster.
# You can either use an API key or a set of cluster username and password joined by a colon.
TOKEN = '261e4b2aa8e7db717791e0db55c82ec2c7c42593c07c9f60e9ba70bdbcbfa5a12f0133c7aead468155c5b3317cc8fc543266a1fd'

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
# Use the WebBaseLoader to load specified web pages into documents
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

extracted_data = load_pdf("data/")

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

text_chunks = text_split(extracted_data)
print(len(text_chunks))

# Split the documents into smaller chunks
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
#all_splits = text_splitter.split_documents(docs)

56


In [9]:
from sentence_transformers import SentenceTransformer


#model_name=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
#text_sentences = [doc.page_content for doc in text_chunks]
def download_hugging_face_embeddings():
    # model_name=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    # embeddings = HuggingFaceEmbeddings(model_name.encode(text_sentences))

    #Initialize the HuggingFaceEmbeddings with the model name
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    # Generate embeddings using the embedding function
    #embeddings = embedding_function.embed_documents(text_chunks)
    
    return embeddings

c:\Users\jaisw\.conda\envs\tknoChatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:

embeddings = download_hugging_face_embeddings()
print("Type of embeddings:", type(embeddings)) 

C:\Users\jaisw\AppData\Local\Temp\ipykernel_14320\2776484176.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Type of embeddings: <class 'langchain_community.embeddings.huggingface.HuggingFaceEmbeddings'>


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result=embeddings.embed_query("hello world")
print(len(query_result))

384


In [18]:

connection_args = { 'uri': URI, 'token': TOKEN }

ids = [f"doc_{i}" for i in range(len(text_chunks))]

vector_store = Zilliz.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    collection_name=COLLECTION_NAME,
    connection_args=connection_args,
    drop_old=True,
    ids=ids
)

In [19]:
query = "wha is techno planet?"
docs = vector_store.similarity_search(query)

print(len(docs))

4


In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:

#user_input=input(f"Input Prompt:")
result=qa({"query": "What is Techno planet"})
print("Response : ", result["result"])

C:\Users\jaisw\AppData\Local\Temp\ipykernel_14320\1538846190.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": "What is Techno planet"})


Response :  Techno Planet is an IT security assessment and penetration testing company that provides various services such as CCTV system, access control system, intrusion detection system, security central & monitoring rooms, PA/GA system, MATV/SMATV/IPTV system, automatic doors system, gate boom barrier & bollard system.
